# Student Information

Wangyang He


UIN 625004872


Nickname: heswaggy

Project submission 2

In [1]:
import os
import numpy as np
import cv2
import json
import keras
from keras.models import load_model
from tqdm.notebook import tqdm

In [2]:
!pip install --upgrade youtube_dl

     |████████████████████████████████| 1.9MB 13.9MB/s 


In [3]:
test_videos = '/content/applyvideos'
os.mkdir(test_videos)

In [16]:
import datetime
import youtube_dl, subprocess

#these are the videos link to test, uncomment to download them
urls = ['https://www.youtube.com/watch?v=UqQqhaBSTAg',
         # 'https://www.youtube.com/watch?v=dO8anUw4zvY',
         #'https://www.youtube.com/watch?v=eAfukUmsbjs',
         #'https://www.youtube.com/watch?v=kdt_jXB95Qk',
        #'https://www.youtube.com/watch?v=O-Ys7Q0rf34'
         ]

for i, id in enumerate(urls):
  #save videos to the folder we created above
  true = os.path.join(test_videos,"video.24.mp4".format(i))
  #download videos
  with youtube_dl.YoutubeDL({'format': 'best'}) as ydl:
    try:
      info = ydl.extract_info(id, download=False)
      video = info['entries'][0] if 'entries' in info else info
    except:
      continue
  #parameters
  url = video['url']
  starttime = datetime.timedelta(seconds=0)
  clip_length = datetime.timedelta(seconds=1000)
  subprocess.call(['ffmpeg', '-i', url, '-ss', str(starttime), '-t', str(clip_length), 
                   '-c:v', 'copy', '-c:a', 'copy', true,]) 
  print("finished downloading")

[youtube] UqQqhaBSTAg: Downloading webpage
finished downloading


In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [11]:
model = load_model('/content/drive/MyDrive/CSCE636/fall_VGG16_sub2.h5')
#loading trained model
print("model loaded")

model loaded


In [17]:
def check_model(img, video, model): #this function will check the video with our model, and predict accruacy to produce JSON
    #get the video itself
    capture = cv2.VideoCapture(img)
    # get the video frame per second
    fps = capture.get(5) 
    # get all frames in the video
    video_frame = int(capture.get(7)) 
    # get video length in seconds
    length = int(video_frame/fps) 
    # get the number of predictions our model will make on the test video
    num_pred = int(length/2) 
    # frame per prediction
    fpp = video_frame // num_pred
    # length per frame
    dis = fpp // 15
    #count the frames for each 2 second clip
    count = 0
    pred_count = 1
    #some parameters to use in the for loop
    pred_time = []
    final_frame = []
    clips = []
    tmp = []

    for i in tqdm(range(video_frame)): #iterating over all the frames of the video
            #using openCV
            ret, frame = capture.read()
            count = count + 1

            if count == dis:
                count = count * 0
                frame=cv2.resize(frame,(150,150),interpolation=cv2.INTER_AREA)
                tmp.append(frame)
                final_frame.append(frame)
                
            if i == ((pred_count * fpp) - 1):
                inpt = np.array(tmp)
                inpt = inpt.astype('float32')
                inpt -= np.mean(inpt)
                inpt /= np.max(inpt)
                tmp = []
                data = np.expand_dims(inpt, axis=0)
                pred = model.predict(data, batch_size = 1) # verbose = 1
                rate = pred[0][1] #prediction score stored
                time_init = (pred_count - 1) / 0.5 #start time
                time_end = pred_count / 0.5  # end time
                #produce JSON file
                if rate > 0.7: #if prediction score better than 70%, add in the JSON file
                    time_result = ['Target: ' +  str(time_init) + 'secs - ' + str(time_end) + 'secs.']
                    json_format = {
                        'videoId': "UqQqhaBSTAg",
                        'type': "segment",
                        'startTime': np.float(time_init),
                        'endTime': np.float(time_end),
                        'observer': 'CSCE636Spring2021-heswaggy-2',
                        'isHuman': False,
                        'confirmedBySomeone': False,
                        'rejectedBySomeone': False,
                        'observation': {
                            'label': 'fall',
                            'labelConfidence': np.float(rate)
                        }
                    }
                    clips.append(json_format)
                else:
                    time_result = ['Others: ' +  str(time_init) + 'secs - ' + str(time_end) + 'secs.']
                pred_count += 1
                pred_time.append(time_result)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    capture.release()
    cv2.destroyAllWindows()

    with open(video, 'w', encoding='utf-8') as f:
        json.dump(clips, f, indent=4)
    return final_frame

print("finished")

finished


In [18]:
final_frame = check_model('/content/applyvideos/video.24.mp4', 'video_24.json', model)

In [110]:
!cp -a '/content/applyvideos/' '/content/drive/MyDrive/CSCE636/sub1testvideos'